In [ ]:
# Starter imports
import os, threading, time, mmap

# TODO: add demos for threads, locks, and simple mmap usage

## Threads, Locks, and Scheduling
- Threads share memory; use locks to guard shared state.
- Use `time.sleep` to simulate work and observe interleaving.

In [ ]:
import mmap, tempfile, os

with tempfile.TemporaryFile() as f:
    f.write(b"hello os mmap demo")
    f.flush()
    f.seek(0)
    with mmap.mmap(f.fileno(), length=0, access=mmap.ACCESS_READ) as mm:
        snippet = mm[:5]
snippet

In [ ]:
import threading, time

def increment(counter, lock, rounds=50_000):
    for _ in range(rounds):
        with lock:
            counter[0] += 1

counter = [0]
lock = threading.Lock()
threads = [threading.Thread(target=increment, args=(counter, lock)) for _ in range(4)]
for t in threads: t.start()
for t in threads: t.join()
counter[0]

## Processes vs Threads (quick note)
- Threads share memory (need locks); processes isolate memory (safer, more overhead).
- For CPU-bound work, processes avoid the GIL; for I/O-bound, threads/async are fine.

In [ ]:
import concurrent.futures, math, time

def cpu_task(n=500_000):
    return sum(i*i for i in range(n))

def bench(pool_cls, workers=4):
    t0 = time.perf_counter()
    with pool_cls(max_workers=workers) as pool:
        list(pool.map(cpu_task, [200_000]*workers))
    return round(time.perf_counter() - t0, 3)

{
    "threads": bench(concurrent.futures.ThreadPoolExecutor),
    "processes": bench(concurrent.futures.ProcessPoolExecutor),
}

## Scheduling and I/O Notes
- Preemptive schedulers time-slice threads; `sleep` yields voluntarily.
- CPU-bound work benefits from fewer runnable threads; I/O-bound can multiplex many.
- Disk/network I/O: use buffers; `fsync`/`flush` when durability matters; async I/O to avoid blocking.

In [ ]:
# Buffered vs flushed file write demo
import os, tempfile, time

data = b"x" * 1024
with tempfile.NamedTemporaryFile(delete=False) as f:
    path = f.name
    f.write(data)
    f.flush()  # buffered flush
    os.fsync(f.fileno())  # force to disk

size = os.path.getsize(path)
os.remove(path)
size